#### Import Pandas

In [1]:
import pandas as pd

#### For Visualizations

In [2]:
import cufflinks as cf; cf.go_offline()
import plotly_express as px

#### Load Data

In [3]:
ad_recs_annotated = pd.read_csv('data/ad_annotations.csv')
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider
64896,2021-week_37,AD08V8YTD8,Puentes Company,IN,no,True,Yield,UserPopY
102163,2021-week_40,AD0887VR8N,Search Million Culture,DE,yes,NaN,Qality,DNNQ
152885,2021-week_43,AD08CXWY8S,RelayFoods,JP,yes,NaN,Qality,UserPopQ
130860,2021-week_42,AD0888P888,RelayFoods,DE,no,False,Qality,DNNQ
235686,2021-week_49,AD078Y78PH,Puentes Company,IN,yes,NaN,Yield,UserPopY


#### First, fix typos (for me)

In [6]:
#fixing typos
ad_recs_annotated = ad_recs_annotated.rename(columns={'is_sever': 'is_severe'})
ad_recs_annotated['recommendation_type'].mask(ad_recs_annotated['recommendation_type'] == 'Qality', 'Quality', inplace=True)
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_severe,recommendation_type,rec_provider
127821,2021-week_41,AD0988CJZ8,Extreme DA,US,yes,NaN,Yield,RNNY
223125,2021-week_48,AD0988C8K8,RelayFoods,IN,yes,NaN,Quality,UserPopQ
101092,2021-week_40,AD007PDDDK,Search Million Culture,DE,no,False,Quality,DNNQ
112856,2021-week_40,AD007XZU0E,Extreme DA,US,yes,NaN,Yield,DNNY
128790,2021-week_41,AD077P88V8,Search Million Culture,US,yes,NaN,Quality,RNNQ


## Questions

### Data Modifications

1. Add a column with the sample year

In [8]:
ad_recs_annotated['sample_year'] = ad_recs_annotated['week_id'].str.split('-').str[0]
ad_recs_annotated.sample(1)

,week_id,ad_id,requester,region,is_success,is_severe,recommendation_type,rec_provider,sample_year
245317,2021-week_50,AD09FSW6DC,Search Million Culture,DE,yes,NaN,Quality,UserPopQ,2021


2. Add a column with the sample quarter (google how to)

In [9]:
ad_recs_annotated['week_num'] = ad_recs_annotated['week_id'].str[-2:].astype(int)
ad_recs_annotated['quarter'] = ((ad_recs_annotated['week_num']-1) // 13) + 1
ad_recs_annotated.sample(1)

,week_id,ad_id,requester,region,is_success,is_severe,recommendation_type,rec_provider,sample_year,week_num,quarter
246813,2021-week_50,AD07J8YKT6,MoJoe Brewing Company,GB,yes,NaN,Yield,RuleBasedY,2021,50,4


3. Add a new success column with values 1 and 0 for yes and no 

In [10]:
ad_recs_annotated['is_success_flag'] = ad_recs_annotated['is_success'].apply(lambda x: 1 if x=='yes' else 0)
ad_recs_annotated.sample(1)

,week_id,ad_id,requester,region,is_success,is_severe,recommendation_type,rec_provider,sample_year,week_num,quarter,is_success_flag
336069,2022-week_05,AD098C8Y98,RelayFoods,IN,no,False,Quality,UserPopQ,2022,5,1,0


4. Are there any NaNs in the is_sever column? Count how many and create a new column with NaNs filled as False. Check that indeed the new column contaion no NaNs.

In [16]:
print(ad_recs_annotated['is_severe'].isna().value_counts())
ad_recs_annotated['is_severe_no_nans'] = ad_recs_annotated['is_severe'].fillna(False)
print(ad_recs_annotated['is_severe_no_nans'].value_counts())
print(ad_recs_annotated['is_severe_no_nans'].isna().value_counts())

True     304195
False     69593
Name: is_severe, dtype: int64
False    329099
True      44689
Name: is_severe_no_nans, dtype: int64
False    373788
Name: is_severe_no_nans, dtype: int64


5. Capitlize (first letter only) the is_success column (Inplace)

In [17]:
ad_recs_annotated['is_success'] = ad_recs_annotated['is_success'].apply(lambda x: x.capitalize())
ad_recs_annotated['is_success'].sample(1)

364508    Yes
Name: is_success, dtype: object

### Subset selection

for each question, sub-select the data by using the conditional selection (`[]`) and the `.query` API. Use `.shape` on the subselection to obtain the result.  
For example:
`df.query('some_condition').shape`

1. How many requests are there in US region? 

In [18]:
ad_recs_annotated.loc[ad_recs_annotated['region']=='US'].shape[0]

103846

In [19]:
ad_recs_annotated.query("region=='US'").shape[0]

103846

2. How many **successful** requests are there in US region? 

In [20]:
ad_recs_annotated[(ad_recs_annotated['region']=='US') & (ad_recs_annotated['is_success_flag']==1)].shape

(88918, 13)

In [21]:
ad_recs_annotated.query("region=='US' and is_success_flag==1").shape

(88918, 13)

3. How many **successful** requests are there in US region, on week 52?

In [25]:
ad_recs_annotated[(ad_recs_annotated['region']=='US') & (ad_recs_annotated['is_success_flag']==1)\
                  & (ad_recs_annotated['week_num']==52)].shape

(3342, 13)

In [27]:
ad_recs_annotated.query("region=='US' and is_success_flag==1 and week_num==52").shape

(3342, 13)

4. How many requests where delivered by DNNQ, RNNY and UserPopQ together? 

In [28]:
ad_recs_annotated[ad_recs_annotated['rec_provider'].isin(['DDNQ', 'RRNY', 'UserPopQ'])].shape

(69937, 13)

In [29]:
ad_recs_annotated.query("rec_provider in ('DDNQ', 'RRNY', 'UserPopQ')").shape

(69937, 13)

5. How many requests where delivered by rule based providers? 

In [30]:
ad_recs_annotated[(ad_recs_annotated['rec_provider'].str.contains('RuleBased', regex=False))].shape

(28336, 13)

In [34]:
ad_recs_annotated.query('rec_provider.str.contains("RuleBased")', engine='python').shape

(28336, 13)

6. Select only the requests from 2021 - How many are there? 

In [35]:
print(ad_recs_annotated[ad_recs_annotated['sample_year']=='2021'].shape)
print(ad_recs_annotated.query("sample_year == '2021'").shape)

(271615, 13)
(271615, 13)


7. Select the reqeusts from week 30 to week 40 - How many are there? 

In [36]:
print(ad_recs_annotated[ad_recs_annotated['week_num'].between(30,40)].shape)
print(ad_recs_annotated.query("week_num >= 30 and week_num <= 40").shape)

(115051, 13)
(115051, 13)


8. Select all the data that comes from the most popular Ad

In [43]:
print(ad_recs_annotated['ad_id'].value_counts()[:1])
print(ad_recs_annotated.ad_id.value_counts().idxmax())
print(ad_recs_annotated.ad_id.mode())
#not sure how this can be done with query method

AD07PFFMP9    247
Name: ad_id, dtype: int64
AD07PFFMP9
0    AD07PFFMP9
dtype: object


9. Select all the data that comes from the least popular provider

In [47]:
#least_pop = ad_recs_annotated['rec_provider'].value_counts()[-1:].index.tolist()[0]
least_pop = ad_recs_annotated.rec_provider.value_counts().idxmin()
print(least_pop)
print(ad_recs_annotated[ad_recs_annotated['rec_provider']==least_pop].shape)
print(ad_recs_annotated.query("rec_provider == 'UserPopSelectionY'").shape)

UserPopSelectionY
(21, 13)
(21, 13)


10. Select the data in which is_sever is None. How big is it? 

In [48]:
print(ad_recs_annotated[ad_recs_annotated['is_severe'].isna()].shape)
ad_recs_annotated.query("is_severe != is_severe").shape

(304195, 13)


(304195, 13)

11. Select the data in which the requester is a 3 word company

In [49]:
print(ad_recs_annotated[ad_recs_annotated['requester'].apply(lambda x: len(x.split())==3)].shape)

(118141, 13)


12. Select the data in which the requester is a 3 word company, and the ad_id has the letter 6 in it

In [50]:
ad_recs_annotated[(ad_recs_annotated['requester'].apply(lambda x: len(x.split())==3))\
                  & (ad_recs_annotated['ad_id'].str.contains('6', regex=False))].shape

(24197, 13)

13. Select the data in which the requester is a 3 word company, and the multiplication of all the numbers in the ad_id is bigger than 30

In [99]:
import functools

def get_id_prod(exp):
    a = ''.join(x for x in exp if x.isdigit())
    b = [int(i) for i in a]
    prod = functools.reduce (lambda x, y: x * y, b)
    return prod

In [122]:
ad_recs_annotated[(ad_recs_annotated.ad_id.apply(lambda x: get_id_prod(x)) > 30) \
                  & (ad_recs_annotated['requester'].apply(lambda x: len(x.split())==3))].shape

(983, 13)

### Analysis

1. How many weeks available in the data? Are there any missing weeks? 

In [128]:
sorted(ad_recs_annotated.week_id.unique())

['2021-week_33',
 '2021-week_34',
 '2021-week_35',
 '2021-week_36',
 '2021-week_37',
 '2021-week_38',
 '2021-week_39',
 '2021-week_40',
 '2021-week_41',
 '2021-week_42',
 '2021-week_43',
 '2021-week_44',
 '2021-week_45',
 '2021-week_46',
 '2021-week_47',
 '2021-week_48',
 '2021-week_49',
 '2021-week_50',
 '2021-week_51',
 '2022-week_01',
 '2022-week_02',
 '2022-week_03',
 '2022-week_04',
 '2022-week_05',
 '2022-week_06',
 '2022-week_52']

In [129]:
ad_recs_annotated.week_id.value_counts().sort_index()

2021-week_33    14356
2021-week_34    14371
2021-week_35    14373
2021-week_36    14390
2021-week_37    14392
2021-week_38    14347
2021-week_39    14416
2021-week_40    14406
2021-week_41    14380
2021-week_42    14376
2021-week_43    14367
2021-week_44    14299
2021-week_45    14440
2021-week_46    14350
2021-week_47    14402
2021-week_48    14404
2021-week_49    14337
2021-week_50    14404
2021-week_51    12805
2022-week_01    14385
2022-week_02    14423
2022-week_03    14317
2022-week_04    14367
2022-week_05    14362
2022-week_06    14389
2022-week_52    15930
Name: week_id, dtype: int64

In [130]:
df2 = ad_recs_annotated.groupby(['week_num', 'sample_year']).size().to_frame()
df2.drop([0], axis=1, inplace=True)
df2.groupby('sample_year').size()

sample_year
2021    19
2022     7
dtype: int64

2. How many regions available in the data? Are there any missing weeks per region?

In [131]:
df2 = ad_recs_annotated[['region','week_id']]
df2.groupby('region').nunique()

,week_id
region,
DE,26
GB,26
IN,26
JP,25
US,26


In [346]:
rdf = ad_recs_annotated.groupby('region').week_num.unique().to_frame()
rdf

,week_num
region,
DE,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4..."
GB,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4..."
IN,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4..."
JP,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4..."
US,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4..."


In [349]:
lol = ad_recs_annotated.groupby('region').week_num.unique().tolist()
lols = [set(l) for l in lol]

In [350]:
def comp_sets(s):
    diff_list = []
    for i in lols:
        diff_list.append(s.symmetric_difference(i))
    return diff_list

In [352]:
rdf['diff'] = rdf.week_num.apply(lambda row: comp_sets(set(row)))
rdf

,week_num,diff
region,,
DE,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[{}, {}, {}, {51}, {}]"
GB,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[{}, {}, {}, {51}, {}]"
IN,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[{}, {}, {}, {51}, {}]"
JP,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[{51}, {51}, {51}, {}, {51}]"
US,"[33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 4...","[{}, {}, {}, {51}, {}]"


3. How many ads are being sent to annotation in each region per week?

In [353]:
#avg amount of ads per week
ad_recs_annotated.sample(5)
df2 = ad_recs_annotated.groupby('region').nunique().reset_index()[['region', 'week_id', 'ad_id']]
df2['ads_per_week'] = df2['ad_id']/df2['week_id']
df2

,region,week_id,ad_id,ads_per_week
0,DE,26,54650,2101.923077
1,GB,26,64135,2466.730769
2,IN,26,61297,2357.576923
3,JP,25,38048,1521.920000
4,US,26,87323,3358.576923


In [356]:
#sum of ads each each week
per_week = ad_recs_annotated.groupby(['week_id', 'region']).size()
per_week = per_week.unstack('region')
per_week.head()

region,DE,GB,IN,JP,US
week_id,,,,,
2021-week_33,2395.0,3180.0,3196.0,1611.0,3974.0
2021-week_34,2360.0,3187.0,3219.0,1599.0,4006.0
2021-week_35,2393.0,3209.0,3209.0,1580.0,3982.0
2021-week_36,2401.0,3209.0,3213.0,1589.0,3978.0
2021-week_37,2404.0,3181.0,3183.0,1644.0,3980.0


4. How many None values are there in is_sever column?

In [357]:
ad_recs_annotated.is_severe.isna().value_counts()

True     304195
False     69593
Name: is_severe, dtype: int64

5. Are ads being sent more than one time in a given week? what is the ad_id that was sent the most in a given week? (e.g. ad X where sent 50 times in week Y)

In [364]:
df3 = ad_recs_annotated[['week_id', 'ad_id', 'recommendation_type']]
df3_counts = df3.groupby(['week_id', 'ad_id']).count()
df3_counts[df3_counts['recommendation_type']>1]
df3_counts.sort_values(by=['recommendation_type'], ascending=False).iloc[0]
#df3_counts.recommendation_type.idxmax()

recommendation_type    81
Name: (2021-week_40, AD098SWYF6), dtype: int64

6. Are there ads that are being sent in more than one week?
   1. Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
   2. What are the weeks that the ad from above was sent in? 
   3. Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.

In [366]:
df3_counts2 = df3[['week_id', 'ad_id']].groupby(['ad_id']).count()

req_ad = df3_counts2[df3_counts2['week_id']>1].reset_index()\
.sort_values(by='week_id', ascending=False)['ad_id'].iloc[0]

print(f'the ad that was sent in most amount of weeks is {req_ad}')

the ad that was sent in most amount of weeks is AD07PFFMP9


In [367]:
print(ad_recs_annotated[ad_recs_annotated['ad_id']==req_ad]['week_id'])

7031      2021-week_33
7032      2021-week_33
20010     2021-week_34
20011     2021-week_34
35214     2021-week_35
              ...     
363533    2022-week_52
363534    2022-week_52
363535    2022-week_52
363536    2022-week_52
365817    2022-week_52
Name: week_id, Length: 247, dtype: object


In [368]:
df_count_success = ad_recs_annotated[['week_id', 'ad_id', 'is_success']].groupby('ad_id').nunique()
df_count_success[(df_count_success['week_id']>=2) & (df_count_success['is_success']>1)].iloc[0]
ad_recs_annotated[ad_recs_annotated['ad_id']=='AD00006IDK']

,week_id,ad_id,requester,region,is_success,is_severe,recommendation_type,rec_provider,sample_year,week_num,quarter,is_success_flag,is_severe_no_nans
10424,2021-week_33,AD00006IDK,Extreme DA,US,Yes,NaN,Yield,DNNY,2021,33,3,1,False
188039,2021-week_46,AD00006IDK,RelayFoods,DE,No,True,Quality,DNNQ,2021,46,4,0,True


7. When is_sever is None, what is the number of successful requests? What is the number of non-successful requests? What do you learn from it about the reason for Nones in the first place? 

In [369]:
print(ad_recs_annotated[ad_recs_annotated['is_severe'].isna()]['is_success_flag'].value_counts())
print('NaN in is_severe should only appear in case of a not successful recommendation')

1    304189
0         6
Name: is_success_flag, dtype: int64
NaN in is_severe should only appear in case of a not successful recommendation


8. Per each region, What is the Quality/Yield traffic proportion WoW?

In [428]:
df_wow = ad_recs_annotated[['region','week_id', 'recommendation_type']]
df_wow['is_quality'] = df_wow.recommendation_type.apply(lambda x: 1 if x=='Quality' else 0)
df_wow['is_yield'] = df_wow.recommendation_type.apply(lambda x: 1 if x=='Yield' else 0)
df_wow = df_wow.groupby(['region','week_id']).sum()
df_wow['prop'] = (df_wow.is_quality / df_wow.is_yield)
df_wow.sample(5)

,,is_quality,is_yield,prop
region,week_id,,,
DE,2022-week_03,1881,497,3.784708
US,2021-week_42,1738,2266,0.766990
DE,2022-week_52,1676,706,2.373938
IN,2022-week_01,1333,1851,0.720151
US,2021-week_50,1865,2126,0.877234


9. How many different reqeusters are there?

In [429]:
ad_recs_annotated['requester'].nunique()

36

10. Which are the top 5 requesters per region?

In [449]:
ad_recs_annotated.groupby(['region','requester'])['is_success_flag'].count()\
.sort_values(ascending=False).groupby('region').nlargest(5)

region  region  requester             
DE      DE      Search Million Culture    23657
                RelayFoods                17313
                MoJoe Brewing Company      8616
                iDreamsky Technology       3542
                SOLOMO365                  2435
GB      GB      Search Million Culture    28002
                RelayFoods                15920
                Fancy                     13678
                MoJoe Brewing Company      9530
                iDreamsky Technology       3919
IN      IN      Puentes Company           29427
                Search Million Culture    21365
                RelayFoods                17352
                MoJoe Brewing Company      7191
                iDreamsky Technology       1995
JP      JP      RelayFoods                24574
                Search Million Culture     7906
                MoJoe Brewing Company      3549
                LocalVox Media             1724
                Tab Solutions              1366
U

11. Which are the different rec_proivders?

In [431]:
ad_recs_annotated['rec_provider'].value_counts()

DNNQ                 117424
UserPopQ              69937
DNNY                  45116
UserPopY              38600
RuleBasedY            28154
RNNQ                  20983
ManualQ               13844
RNNY                  12732
XGBQ                  12250
XGBY                   4276
BooksY                 4150
UserPopSelectionQ      2417
ManualY                1982
BooksQ                 1720
RuleBased               182
UserPopSelectionY        21
Name: rec_provider, dtype: int64

12. Are there different rec providers per region?

In [435]:
ad_recs_annotated.groupby('region')['rec_provider'].nunique()

region
DE     9
GB    12
IN     6
JP     9
US    14
Name: rec_provider, dtype: int64

13. Are there different rec providers per rec type?

In [437]:
ad_recs_annotated.groupby(['recommendation_type', 'rec_provider'])['rec_provider'].nunique()

recommendation_type  rec_provider     
Quality              BooksQ               1
                     DNNQ                 1
                     ManualQ              1
                     ManualY              1
                     RNNQ                 1
                     RuleBased            1
                     UserPopQ             1
                     UserPopSelectionQ    1
                     XGBQ                 1
Yield                BooksY               1
                     DNNY                 1
                     ManualQ              1
                     ManualY              1
                     RNNY                 1
                     RuleBasedY           1
                     UserPopSelectionY    1
                     UserPopY             1
                     XGBY                 1
Name: rec_provider, dtype: int64

14. What are the notation rules distinguishing between quality vs yielding providers?

In [438]:
#from the previous q we can see that apart from manual, all yield end with "Y",
#and all quality end with "Q"

15. Which is the most successful region of operation?

In [442]:
ad_recs_annotated.groupby('region')['is_success_flag'].sum().idxmax()

'US'

16. Which is the most successful rec provider per each region?

In [448]:
ad_recs_annotated.groupby(['region', 'rec_provider'])['is_success_flag'].sum()\
.sort_values(ascending=False).groupby('region').nlargest(1)

region  region  rec_provider
DE      DE      DNNQ            27215
GB      GB      DNNQ            34681
IN      IN      UserPopQ        30407
JP      JP      DNNQ            20113
US      US      DNNY            25705
Name: is_success_flag, dtype: int64